In [5]:
from numpy import reshape
import seaborn as sns
import pandas as pd  
import torch
from os import listdir, walk
from os.path import isfile, join
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import soundfile as sf
import numpy as np
from IPython.display import display, Audio
import librosa
import random
def ms2samples(time, rate):
    return int((time/1000)*rate)

def samples2ms(sample,rate):
    return int((sample/rate)*1000)
    
def graph_n_play(y,sr):
    plt.figure(figsize=(14, 5))
    librosa.display.waveplot(y, sr=sr)
    plt.show()
    ipd.display(ipd.Audio(y,rate = sr))
def linear_n_play(audio_path):
    # y, sr = sf.read(audio_path)
    y, sr = librosa.load(audio_path, sr=None)
    fig, ax = plt.subplots()
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    img = librosa.display.specshow(D, y_axis='linear', x_axis='time',
                                   sr=sr, ax=ax)
    ax.set(title='Linear-frequency power spectrogram')
    ax.label_outer()
    display(Audio(y,rate=sr))
    plt.show()
def recursive_list_files(path, file_extension=".wav"):
    """Recursively lists all files in a directory and its subdirectories"""
    files = []
    for dirpath, dirnames, filenames in walk(path):
        for filename in filenames:
            if filename.endswith(file_extension):
                files.append(join(dirpath, filename))
    return files

In [7]:
# list all file in real data

real_data = recursive_list_files("/dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs")
len(real_data)

213989

In [8]:
# list all file in fake data
fake_data = recursive_list_files("/dataa/Dataset/Korean_Fake/Wav_voice")
len(fake_data)

32600

In [10]:
df = pd.DataFrame(columns=["spid","utt","path","label"])
for i in tqdm(range(len(real_data))):
    df = pd.concat([df,pd.DataFrame({"spid":real_data[i].split("/")[-2],"utt":real_data[i].split("/")[-1],"path":real_data[i],"label":"real"},index=[0])],axis=0)

for i in tqdm(range(len(fake_data))):
    df = pd.concat([df,pd.DataFrame({"spid":fake_data[i].split("/")[-2].split("_")[0],"utt":fake_data[i].split("/")[-1],"path":fake_data[i],"label":"fake"},index=[0])],axis=0)
    
df.headff

100%|██████████| 32600/32600 [10:06<00:00, 53.72it/s]


<bound method NDFrame.head of           spid                    utt  \
0   05MSTK0045   05MSTK0045_06647.wav   
0   05MSTK0045   05MSTK0045_06821.wav   
0   05MSTK0045   05MSTK0045_07527.wav   
0   05MSTK0045   05MSTK0045_07359.wav   
0   05MSTK0045   05MSTK0045_07426.wav   
..         ...                    ...   
0   07MAJM0028  07MAJM0028_000019.wav   
0   07MAJM0028  07MAJM0028_000003.wav   
0   07MAJM0028  07MAJM0028_000101.wav   
0   07MAJM0028  07MAJM0028_000188.wav   
0   07MAJM0028  07MAJM0028_000094.wav   

                                                 path label  
0   /dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs...  real  
0   /dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs...  real  
0   /dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs...  real  
0   /dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs...  real  
0   /dataa/aisrc1/Dataset/Speech/AIHub_Studio/wavs...  real  
..                                                ...   ...  
0   /dataa/Dataset/Korean_Fake/Wav_voice/07

In [27]:
real_df = df[:213989]
fake_df = df[213989:]

In [29]:
real_df = real_df.drop(real_df.sample(frac=.7).index)
real_df.head

In [14]:
# test dataframe
df.loc[(df["spid"]=="07MAJM0028") & (df["label"]=="fake")].sample().values[0]

array(['07MAJM0028', '07MAJM0028_000169.wav',
       '/dataa/Dataset/Korean_Fake/Wav_voice/07MAJM0028_Voice_data/07MAJM0028_000169.wav',
       'fake'], dtype=object)

In [15]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [18]:

train_df.to_csv("/dataa/aisrc1/Dataset/Speech/AIHub_Studio/train.csv",index=False)

((8243, 4), (53405, 4))